## Embedding layer

In [ ]:
import numpy as np
from keras.layers import Input, Dense
from keras.layers.embeddings import Embedding
from keras.models import Model

max_sequence = 5
embedding_dims = 10

# Size of the vocabulary. The assumption is that indexing starts with 0
# and is consequtive.
vocab_size = 3

inputs = Input(shape=(max_sequence,), dtype='int32') # each X_i is a sequence of 'max_sequence' integers
outputs = Embedding(vocab_size, embedding_dims, input_length=max_sequence)(inputs)

model = Model([inputs], [outputs])
model.compile(optimizer='sgd', loss='mse')

model.predict( np.array([[0,1,1,1,2]]) )

# An embedding layer is like a lookup table. The values in the input 
# vector are used as indices in the internal weights matrix.

## Lambda layer
Simple way to add functionality to a model. Best used for stateless functions. For stateful functions it is better to implement a separate layer.

In [5]:
import numpy as np
from keras.layers import Input, Concatenate, Lambda, Embedding, Average, Dense, Add
from keras.models import Model


inputs = Input(shape=(4,4), dtype='float32')
first_row = Lambda(lambda x: x[:,0,:])(inputs)
first_column = Lambda(lambda x: x[:,:,0])(inputs)
doubled = Lambda(lambda x: x*2.0)(inputs)

model = Model(inputs=inputs, outputs=[first_row, first_column, doubled])
model.compile(optimizer='sgd', loss='mse')

x = np.array([
    [
    [1,1,1,1],
    [2,2,2,2],
    [3,3,3,3],
    [4,4,4,4],        
    ]
], dtype=np.float32)

x,y,z = model.predict(x)
print(x)
print(y)
print(z)

[[1. 1. 1. 1.]]
[[1. 2. 3. 4.]]
[[[2. 2. 2. 2.]
  [4. 4. 4. 4.]
  [6. 6. 6. 6.]
  [8. 8. 8. 8.]]]


## Merge layer: concatenate, average and sum

In [2]:
import numpy as np
from keras.layers import Input, Concatenate, Lambda, Embedding, Average, Dense, Add
from keras.models import Model


inputs = Input(shape=(4,4), dtype='float32')
word_vector_rows = [Lambda(lambda x: x[:,i,:])(inputs) for i in range(win_size)]
concat_out = Concatenate()(word_vector_rows)
avg_out = Average()(word_vector_rows)
sum_out = Add()(word_vector_rows)


model = Model(inputs=inputs, outputs=[concat_out, avg_out, sum_out])

model.compile(optimizer='sgd', loss='mse')

x = np.array([
    [
    [1,1,1,1],
    [2,2,2,2],
    [3,3,3,3],
    [4,4,4,4],        
    ]
], dtype=np.float32)

print(x.shape)

concat_out_val, avg_out_val, sum_out_val = model.predict(x)
print('concat_out', concat_out_val)
print('avg_out', avg_out_val)
print('sum_out', sum_out_val)

(1, 4, 4)
concat_out [[1. 1. 1. 1. 2. 2. 2. 2. 3. 3. 3. 3. 4. 4. 4. 4.]]
avg_out [[2.5 2.5 2.5 2.5]]
sum_out [[10. 10. 10. 10.]]
